In [10]:
%%writefile mapper4.py

import sys
import re

def read_stopwords(file_path):
    return set(word.strip() for word in open(file_path))

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

stop_words = read_stopwords('/datasets/stop_words_en.txt')

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        word = word.strip()
        if word and len(word) > 1 and word.isalpha():
            word = word.lower();
            if word in stop_words:
                continue
            letterList = list(word)
            letterList.sort()
            #letterSet = set(letterList)
            if len(set(letterList)) > 1:
                print ("%s,%s\t%d" % (''.join(letterList), word, 1))
            

Overwriting mapper4.py


In [7]:
%%writefile reducer4.py

import sys

current_anagram = None
current_word = None
word_count = 0

for line in sys.stdin:
    try:
        a_w, count = line.strip().split('\t', 1)
        anagram, word = a_w.split(',', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_word == word:
        word_count += count
    else:
        if current_anagram:
            if current_word:
                print ("%s,%s\t%d" % (current_anagram, current_word, word_count))
        word_count = count
        current_anagram = anagram
        current_word = word

if current_anagram and current_word:
    print ("%s,%s\t%d" % (current_anagram, current_word, word_count))

Overwriting reducer4.py


In [12]:
%%writefile reducer4a.py

import sys

current_anagram = None
current_word = None
total_count = 0
word_list = []


for line in sys.stdin:
    try:
        a_w, count = line.strip().split('\t', 1)
        anagram, word = a_w.split(',', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_anagram and current_anagram == anagram and current_word == word:
        total_count += count
        continue
    if current_anagram and current_anagram == anagram:
        word_list.append(word)
        total_count += count    
        current_word = word
        continue
    if current_anagram:
        if current_word:  
            word_set = set(word_list) 
            if len(word_set) > 1:
                word_list = list(word_set)
                word_list.sort()
                print ("%d\t%d\t%s" % (total_count, len(word_list), ','.join(word_list)))
    total_count = count
    word_list = [ word ]
    current_anagram = anagram
    current_word = word

if current_anagram and current_word:
    word_set = set(word_list) 
    if len(word_set) > 1:
        word_list = list(word_set)
        word_list.sort()
        print ("%d\t%d\t%s" % (total_count, en(word_list), ','.join(word_list)))

Overwriting reducer4a.py


In [13]:
%%bash

OUT_DIR="wc_result_"$(date +"%s%6N")
NUM_REDUCERS=4
OUT_DIR2="anagram_result_"$(date +"%s%6N")

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null
hdfs dfs -rm -r -skipTrash ${OUT_DIR2} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper4.py,reducer4.py,/datasets/stop_words_en.txt \
    -mapper "python mapper4.py" \
    -reducer "python reducer4.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 
    
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Anagrams" \
    -D mapreduce.job.reduces=1 \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -files reducer4a.py \
    -mapper "cat" \
    -reducer "python reducer4a.py" \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
    -input ${OUT_DIR}/part* \
    -output ${OUT_DIR2} > /dev/null

hdfs dfs -cat ${OUT_DIR2}/part-00000 | grep "english,"


7825	5	english,helsing,hesling,shengli,shingle


rm: `wc_result_1532201367395122': No such file or directory
rm: `anagram_result_1532201367396105': No such file or directory
18/07/21 19:29:35 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/07/21 19:29:36 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/07/21 19:29:37 INFO mapred.FileInputFormat: Total input files to process : 1
18/07/21 19:29:39 INFO mapreduce.JobSubmitter: number of splits:2
18/07/21 19:29:39 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1532183761073_0015
18/07/21 19:29:40 INFO impl.YarnClientImpl: Submitted application application_1532183761073_0015
18/07/21 19:29:40 INFO mapreduce.Job: The url to track the job: http://3474399ea4a6:8088/proxy/application_1532183761073_0015/
18/07/21 19:29:40 INFO mapreduce.Job: Running job: job_1532183761073_0015
18/07/21 19:29:52 INFO mapreduce.Job: Job job_1532183761073_0015 running in uber mode : false
18/07/21 19:29:52 INFO mapreduce.Job:  map 0% reduce 0%
18/07/21 19: